# WSI - ćwiczenie 2
## Algorytmy ewolucyjne i genetyczne
## Yaroslav Harbar

Głównym celem tego ćwiczenia było zaimplementować algorytm genetyczny z selekcją ruletkową, krzyżowaniem jednopunktowym oraz sukcesją generacyjną. Następnie trzeba było zbadać działanie tego algorytmu na problemie z wyląndowaniem rakiety.
Dla rozwiązania tego zadania stworzyłem dwie klasy. Pierwsza klasa **RocketTest** odpowiada za symulację lądowania rakiety oraz obliczenie zysku tej symulacji. Natomiast druga klasa **GeneticAlgorithm** ma zaimplementowany w sobie algorytm ewolucyjny na podstawie którego szukana jest najlepsza symulacja, która dla obliczenia zysku pojedynczej iteracji korzysta z metody *population_test*, która jest zaimplementowana w klasie **RocketTest**.


In [28]:
import numpy as np

class RocketTest:
    def __init__(self, start_height, rocket_weight, start_speed, acceleration):
        self.start_height = start_height
        self.rocket_weight = rocket_weight
        self.start_speed = start_speed
        self.acceleration = acceleration

    def check_rocket_state(self, height):
        if height < 2:
            return True
        return False


    def simulate_individual(self, individual):
        height = self.start_height
        weight = self.rocket_weight
        speed = self.start_speed

        #Add weight of fuel
        fuel = sum(individual)
        weight += fuel

        for gene in individual:
            if gene:
                weight -= 1
                speed += (45/weight)

            speed += self.acceleration
            height += speed

            if 0 <= height < 2 and abs(speed) < 2:
                return 2000 - fuel
            elif height < 0:
                return -1000 - fuel

        return -fuel

    def population_test(self, population):
        test_result = []
        const = 2000

        for individual in population:
            result = self.simulate_individual(individual)

            test_result.append(result + const)

        return test_result

In [21]:
class GeneticAlgorithm:
    def __init__(self, q, population, p_c, p_m, population_size):
        self.population = population
        self.p_c =p_c
        self.p_m = p_m
        self.test = q
        self.population_size = population_size

    def get_parameters(self):
        return (
            self.population,
            self.p_c,
            self.p_m,
            self.test,
            self.population_size
        )

    def find_the_best_individual(self, test_result):
        best_result_index = test_result.index(max(test_result))
        best_result = test_result[best_result_index]
        best_individual = self.population[best_result_index]

        return best_individual, best_result

    def roulette_selection(self, temp_population, test_result, population_size):
        summary = sum(test_result)
        test_result = [result / summary for result in test_result]

        index_list = [i for i in range(len(temp_population))]

        s = np.random.choice(index_list, population_size, replace=True, p=test_result)

        selection_arr = [temp_population[index] for index in s]

        return selection_arr

    def crossover(self, selected_population):
        crossover_arr = list()

        for i in range(1, len(selected_population), 2):
            first_individual = selected_population[i-1]
            second_individual = selected_population[i]

            if np.random.random_sample() < self.p_c:
                index = np.random.randint(1, len(first_individual)-1)

                first_individual_1 = first_individual[:index]
                first_individual_2 = first_individual[index:]
                second_individual_1 = second_individual[:index]
                second_individual_2 = second_individual[index:]

                first_individual = []
                second_individual = []

                first_individual.extend(first_individual_1)
                first_individual.extend(second_individual_2)

                second_individual.extend(second_individual_1)
                second_individual.extend(first_individual_2)

            crossover_arr.append(first_individual)
            crossover_arr.append(second_individual)

        return crossover_arr

    def mutation(self, crossover_arr):
        mutation_arr = list()

        for i in range(len(crossover_arr)):
            individual = crossover_arr[i]

            for j in range(len(individual)):
                if np.random.random_sample() < self.p_m:
                    individual[j] = 1 if individual[j] == 0 else 0

            mutation_arr.append(individual)

        return mutation_arr

    def solve(self, iterations):
        temp_population = self.population

        result = self.test.population_test(temp_population)
        best_individual, best_result = self.find_the_best_individual(result)

        for _ in range(iterations):
            selection_arr = self.roulette_selection(temp_population, result, self.population_size)
            crossover_arr = self.crossover(selection_arr)
            mutation_arr = self.mutation(crossover_arr)

            temp_population = mutation_arr

            result = self.test.population_test(temp_population)
            new_best_individual, new_best_result = self.find_the_best_individual(result)

            if new_best_result >= best_result:
                best_result = new_best_result
                best_individual = new_best_individual

        return best_individual, best_result - 2000

## Ustawienia dla algorytmu genetycznego

Założyłem, że jeden osobnik będzie miał 200 genów i każdy gen odpowiada za działanie silnika w chwili *i*, gdzie 0 - silnik wyłączony, 1 - silnik włączony

Dla bardziej wygodnego badania wpływu hiperparametrów na działanie algorytmu i przeprowadzenia symulacji stworzyłem klasę **Simulation**. W środku tej klasy jest metoda simulation, która w każdej symulacji szuka najlepszego osobnika i wyświetla jego rezultat. Na koniec działania wyświetlany najlepszy osobnik, jego wynik oraz obliczony średni wynik wszystkich symulacji.

In [25]:
class Simulation:
    def __init__(self, rocket_test):
        self.rocket_test = rocket_test

    def set_rocket_test(self, rocket_test):
        self.rocket_test = rocket_test

    def create_random_population(self, population_size):
        p = []

        for _ in range(population_size):
            p.append(np.random.randint(2, size=200))

        return p

    def mean(self, results):
        return sum(results) / len(results)

    def simulate(self, simulation_count, p_c, p_m, population_size, iterations, enable_print=True):
        results = []
        individuals_results = []

        for i in range(simulation_count):
            start_population = self.create_random_population(population_size)

            GA = GeneticAlgorithm(self.rocket_test, start_population, p_c, p_m, population_size)

            individual_result, result  = GA.solve(iterations)

            if enable_print:
                print(f"Simulation {i + 1}: best result - {result}")

            results.append(result)
            individuals_results.append(individual_result)

        best_result_index = results.index(max(results))
        best_result = results[best_result_index]
        best_individual = individuals_results[best_result_index]
        mean = self.mean(results)

        print()
        print(f"Best result: {best_result}")
        print(f"Mean: {mean}")
        print(f"Best individual: {best_individual}")


## Badanie wpływu współczynników krzyżowania i mutacji

Stworzyłem object RT (od Rocket Test) nadając mu wysokość 200, masę rakiety 200, początkowa prędkość 0 oraz przyspieszenie -0.09.

In [26]:
RT = RocketTest(start_height=200, rocket_weight=200, start_speed=0, acceleration=-0.09)
simulation = Simulation(RT)

### Zmiana współczynnika krzyżowania przy stałym współczynniku mutacji

W kolejnych symulacjach prawdopodobieństwo mutacji dałem 5%, natomiast prawdopodobieństwo mutacji zwiększam, oczekując, że ze wzrostem wartości prawdopodobieństwa będęn  lepszy rezultat.

In [13]:
simulation.simulate(simulation_count=25, p_c=0.01, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1918
Simulation 2: best result - 1918
Simulation 3: best result - 1918
Simulation 4: best result - 1918
Simulation 5: best result - 1918
Simulation 6: best result - 1921
Simulation 7: best result - 1919
Simulation 8: best result - 1919
Simulation 9: best result - 1915
Simulation 10: best result - 1915
Simulation 11: best result - 1916
Simulation 12: best result - 1917
Simulation 13: best result - 1922
Simulation 14: best result - 1918
Simulation 15: best result - 1920
Simulation 16: best result - 1917
Simulation 17: best result - 1920
Simulation 18: best result - 1918
Simulation 19: best result - 1917
Simulation 20: best result - 1919
Simulation 21: best result - 1919
Simulation 22: best result - 1919
Simulation 23: best result - 1922
Simulation 24: best result - 1918
Simulation 25: best result - 1917

Best result: 1922
Mean: 1918.32
Best individual: [0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 1 1

In [14]:
simulation.simulate(simulation_count=25, p_c=0.1, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1925
Simulation 2: best result - 1918
Simulation 3: best result - 1920
Simulation 4: best result - 1920
Simulation 5: best result - 1924
Simulation 6: best result - 1927
Simulation 7: best result - 1920
Simulation 8: best result - 1920
Simulation 9: best result - 1918
Simulation 10: best result - 1921
Simulation 11: best result - 1920
Simulation 12: best result - 1924
Simulation 13: best result - 1925
Simulation 14: best result - 1918
Simulation 15: best result - 1922
Simulation 16: best result - 1919
Simulation 17: best result - 1919
Simulation 18: best result - 1920
Simulation 19: best result - 1920
Simulation 20: best result - 1920
Simulation 21: best result - 1922
Simulation 22: best result - 1920
Simulation 23: best result - 1920
Simulation 24: best result - 1919
Simulation 25: best result - 1919

Best result: 1927
Mean: 1920.8
Best individual: [1 1 1 0 1 0 0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0
 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 1 0 

In [6]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1923
Simulation 2: best result - 1922
Simulation 3: best result - 1922
Simulation 4: best result - 1923
Simulation 5: best result - 1922
Simulation 6: best result - 1920
Simulation 7: best result - 1920
Simulation 8: best result - 1924
Simulation 9: best result - 1922
Simulation 10: best result - 1921
Simulation 11: best result - 1921
Simulation 12: best result - 1922
Simulation 13: best result - 1923
Simulation 14: best result - 1923
Simulation 15: best result - 1922
Simulation 16: best result - 1924
Simulation 17: best result - 1923
Simulation 18: best result - 1926
Simulation 19: best result - 1920
Simulation 20: best result - 1923
Simulation 21: best result - 1922
Simulation 22: best result - 1924
Simulation 23: best result - 1923
Simulation 24: best result - 1923
Simulation 25: best result - 1922

Best result: 1926
Mean: 1922.4
Best individual: [1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0
 1 1 0 1 0 0 1 1 0 0 0 0 0 1 1 0 0 

In [7]:
simulation.simulate(simulation_count=25, p_c=0.7, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1924
Simulation 2: best result - 1923
Simulation 3: best result - 1922
Simulation 4: best result - 1924
Simulation 5: best result - 1926
Simulation 6: best result - 1925
Simulation 7: best result - 1925
Simulation 8: best result - 1922
Simulation 9: best result - 1922
Simulation 10: best result - 1921
Simulation 11: best result - 1925
Simulation 12: best result - 1923
Simulation 13: best result - 1924
Simulation 14: best result - 1923
Simulation 15: best result - 1922
Simulation 16: best result - 1923
Simulation 17: best result - 1926
Simulation 18: best result - 1922
Simulation 19: best result - 1922
Simulation 20: best result - 1926
Simulation 21: best result - 1922
Simulation 22: best result - 1923
Simulation 23: best result - 1922
Simulation 24: best result - 1924
Simulation 25: best result - 1923

Best result: 1926
Mean: 1923.36
Best individual: [1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 0 1 1 1 0
 1 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0

In [8]:
simulation.simulate(simulation_count=25, p_c=1, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1922
Simulation 2: best result - 1925
Simulation 3: best result - 1926
Simulation 4: best result - 1922
Simulation 5: best result - 1924
Simulation 6: best result - 1922
Simulation 7: best result - 1925
Simulation 8: best result - 1925
Simulation 9: best result - 1924
Simulation 10: best result - 1923
Simulation 11: best result - 1922
Simulation 12: best result - 1923
Simulation 13: best result - 1921
Simulation 14: best result - 1925
Simulation 15: best result - 1924
Simulation 16: best result - 1922
Simulation 17: best result - 1923
Simulation 18: best result - 1924
Simulation 19: best result - 1924
Simulation 20: best result - 1926
Simulation 21: best result - 1925
Simulation 22: best result - 1924
Simulation 23: best result - 1923
Simulation 24: best result - 1924
Simulation 25: best result - 1923

Best result: 1926
Mean: 1923.64
Best individual: [1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1
 0 0 1 1 0 0 1 0 1 0 1 0 0 1 0 0 0

Jak widać przy wzroście prawdopodobieństwa krzyżowania otrzymany średni wynik jest coraz lepszy. W kolejnych eksperymentach będę korzystać z prawdopodobieństwa krzyżowania równym 50%.

### Zmiana współczynnika mutacji przy stałym współczynniku krzyżowania

W tym przypadku prawdopodobieństwo krzyżowania dałem 50%, biorąc pod uwagę poprzedni eksperyment. Prawdopodobieństwo mutacji będę zwiększać, oczekując, że przy wzroście tej wartości wynik będzie coraz gorszy, a średnia z eksperymentu będzie spadała.

In [10]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0.01, population_size=200, iterations=100)

Simulation 1: best result - 1924
Simulation 2: best result - 1921
Simulation 3: best result - 1922
Simulation 4: best result - 1922
Simulation 5: best result - 1922
Simulation 6: best result - 1921
Simulation 7: best result - 1921
Simulation 8: best result - 1920
Simulation 9: best result - 1925
Simulation 10: best result - 1921
Simulation 11: best result - 1922
Simulation 12: best result - 1922
Simulation 13: best result - 1922
Simulation 14: best result - 1923
Simulation 15: best result - 1923
Simulation 16: best result - 1922
Simulation 17: best result - 1923
Simulation 18: best result - 1922
Simulation 19: best result - 1920
Simulation 20: best result - 1922
Simulation 21: best result - 1920
Simulation 22: best result - 1921
Simulation 23: best result - 1924
Simulation 24: best result - 1922
Simulation 25: best result - 1922

Best result: 1925
Mean: 1921.96
Best individual: [0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 1 1

In [8]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1924
Simulation 2: best result - 1922
Simulation 3: best result - 1923
Simulation 4: best result - 1923
Simulation 5: best result - 1922
Simulation 6: best result - 1922
Simulation 7: best result - 1920
Simulation 8: best result - 1924
Simulation 9: best result - 1923
Simulation 10: best result - 1926
Simulation 11: best result - 1922
Simulation 12: best result - 1923
Simulation 13: best result - 1922
Simulation 14: best result - 1923
Simulation 15: best result - 1922
Simulation 16: best result - 1921
Simulation 17: best result - 1921
Simulation 18: best result - 1922
Simulation 19: best result - 1922
Simulation 20: best result - 1921
Simulation 21: best result - 1921
Simulation 22: best result - 1922
Simulation 23: best result - 1923
Simulation 24: best result - 1921
Simulation 25: best result - 1922

Best result: 1926
Mean: 1922.28
Best individual: [0 1 1 0 1 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1
 1 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0

In [11]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0.1, population_size=200, iterations=100)

Simulation 1: best result - 1923
Simulation 2: best result - 1923
Simulation 3: best result - 1922
Simulation 4: best result - 1923
Simulation 5: best result - 1922
Simulation 6: best result - 1921
Simulation 7: best result - 1922
Simulation 8: best result - 1922
Simulation 9: best result - 1921
Simulation 10: best result - 1921
Simulation 11: best result - 1920
Simulation 12: best result - 1920
Simulation 13: best result - 1919
Simulation 14: best result - 1923
Simulation 15: best result - 1920
Simulation 16: best result - 1923
Simulation 17: best result - 1921
Simulation 18: best result - 1923
Simulation 19: best result - 1923
Simulation 20: best result - 1922
Simulation 21: best result - 1922
Simulation 22: best result - 1922
Simulation 23: best result - 1922
Simulation 24: best result - 1921
Simulation 25: best result - 1921

Best result: 1923
Mean: 1921.68
Best individual: [1 1 0 0 0 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0
 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1

In [12]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0.5, population_size=200, iterations=100)

Simulation 1: best result - 1922
Simulation 2: best result - 1921
Simulation 3: best result - 1921
Simulation 4: best result - 1924
Simulation 5: best result - 1922
Simulation 6: best result - 1925
Simulation 7: best result - 1922
Simulation 8: best result - 1921
Simulation 9: best result - 1923
Simulation 10: best result - 1923
Simulation 11: best result - 1921
Simulation 12: best result - 1921
Simulation 13: best result - 1921
Simulation 14: best result - 1923
Simulation 15: best result - 1920
Simulation 16: best result - 1923
Simulation 17: best result - 1921
Simulation 18: best result - 1922
Simulation 19: best result - 1920
Simulation 20: best result - 1919
Simulation 21: best result - 1921
Simulation 22: best result - 1922
Simulation 23: best result - 1925
Simulation 24: best result - 1922
Simulation 25: best result - 1921

Best result: 1925
Mean: 1921.84
Best individual: [0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1
 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0

In [13]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=1, population_size=200, iterations=100)

Simulation 1: best result - 1921
Simulation 2: best result - 1921
Simulation 3: best result - 1917
Simulation 4: best result - 1919
Simulation 5: best result - 1917
Simulation 6: best result - 1919
Simulation 7: best result - 1919
Simulation 8: best result - 1919
Simulation 9: best result - 1919
Simulation 10: best result - 1918
Simulation 11: best result - 1921
Simulation 12: best result - 1918
Simulation 13: best result - 1922
Simulation 14: best result - 1922
Simulation 15: best result - 1919
Simulation 16: best result - 1922
Simulation 17: best result - 1921
Simulation 18: best result - 1918
Simulation 19: best result - 1920
Simulation 20: best result - 1920
Simulation 21: best result - 1923
Simulation 22: best result - 1919
Simulation 23: best result - 1922
Simulation 24: best result - 1919
Simulation 25: best result - 1918

Best result: 1923
Mean: 1919.72
Best individual: [0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 1
 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0

Analizując powyższe wyniki, możemy zauważyć, że z prawdopodobieństwem mutacji nie jest tak samo, jak z prawdopodobieństwem krzyżowania. Przy zwiększeniu prawdopodobieństwa do 0,05% wynik oraz średnia symulacji rośnie, natomiast już od 0,05% te wartości zaczynają spadać. Dla tego w po dalszych symulacja będę korzystać z prawdopodobieństwa mutacji 0,05%.

## Badanie wpływu wielkości iteracji i populacji


Na początku sprawdzam wpływ wielkości iteracji, zwiększając ją wartość.

In [14]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=10, iterations=1)

Simulation 1: best result - 1907

Best result: 1907
Mean: 1907.0
Best individual: [0 0 1 0 1 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 0 0
 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0
 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1
 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0
 0 0 1 1 1 1 0 0 1 1 0 0 1 0 1]


In [15]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=10, iterations=10)

Simulation 1: best result - 1915

Best result: 1915
Mean: 1915.0
Best individual: [0 1 0 1 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0
 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0
 1 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1
 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1]


In [16]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=10, iterations=100)

Simulation 1: best result - 1916

Best result: 1916
Mean: 1916.0
Best individual: [1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 1
 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0
 0 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1 0 1 1
 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1 0 0 0 0 1
 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0]


In [17]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=10, iterations=1000)

Simulation 1: best result - 1921

Best result: 1921
Mean: 1921.0
Best individual: [1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0
 1 1 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 0 1 1 0
 1 0 0 1 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1
 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 0
 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 0
 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1]


Jak widać przy wzroście ilości iteracji, wynik jest lepszy.

Następnie sprawdzam wpływ wielkości populacji w analogiczny sposób jak po przednio.

In [20]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=10, iterations=100)

Simulation 1: best result - 1918

Best result: 1918
Mean: 1918.0
Best individual: [0 1 0 1 0 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1
 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0
 0 1 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0
 1 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 1
 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0]


In [21]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=100, iterations=100)

Simulation 1: best result - 1921

Best result: 1921
Mean: 1921.0
Best individual: [1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0
 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 1
 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1 0 0 1 0
 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 0
 1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0]


In [22]:
simulation.simulate(simulation_count=1, p_c=0.5, p_m=0.05, population_size=1000, iterations=100)

Simulation 1: best result - 1923

Best result: 1923
Mean: 1923.0
Best individual: [1 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0
 0 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1
 0 1 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0
 0 1 1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0
 1 1 0 0 0 0 1 1 1 1 0 1 1 0 0]


Widać, że przy wzroście wielkości populacji algorytm działa lepiej, ponieważ najlepszy rezultat eksperymentu oraz średnia z eksperymentu rośnie.

## Badanie działania algorytmu zmieniają od razu kilka parametrów

Sprawdzam działanie algorytmu przy braku krzyżowania i przy zmianą prawdopodobieństwa mutacji

In [10]:
simulation.simulate(simulation_count=25, p_c=0, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1919
Simulation 2: best result - 1919
Simulation 3: best result - 1917
Simulation 4: best result - 1916
Simulation 5: best result - 1915
Simulation 6: best result - 1915
Simulation 7: best result - 1920
Simulation 8: best result - 1918
Simulation 9: best result - 1920
Simulation 10: best result - 1916
Simulation 11: best result - 1919
Simulation 12: best result - 1917
Simulation 13: best result - 1916
Simulation 14: best result - 1918
Simulation 15: best result - 1917
Simulation 16: best result - 1918
Simulation 17: best result - 1918
Simulation 18: best result - 1923
Simulation 19: best result - 1915
Simulation 20: best result - 1913
Simulation 21: best result - 1916
Simulation 22: best result - 1920
Simulation 23: best result - 1915
Simulation 24: best result - 1917
Simulation 25: best result - 1921

Best result: 1923
Mean: 1917.52
Best individual: [0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 1 1 1 0 1 0
 0 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 0

In [11]:
simulation.simulate(simulation_count=25, p_c=0, p_m=0.2, population_size=200, iterations=100)

Simulation 1: best result - 1917
Simulation 2: best result - 1919
Simulation 3: best result - 1918
Simulation 4: best result - 1920
Simulation 5: best result - 1917
Simulation 6: best result - 1916
Simulation 7: best result - 1918
Simulation 8: best result - 1917
Simulation 9: best result - 1921
Simulation 10: best result - 1920
Simulation 11: best result - 1919
Simulation 12: best result - 1919
Simulation 13: best result - 1917
Simulation 14: best result - 1918
Simulation 15: best result - 1921
Simulation 16: best result - 1917
Simulation 17: best result - 1919
Simulation 18: best result - 1918
Simulation 19: best result - 1917
Simulation 20: best result - 1917
Simulation 21: best result - 1918
Simulation 22: best result - 1916
Simulation 23: best result - 1916
Simulation 24: best result - 1922
Simulation 25: best result - 1917

Best result: 1922
Mean: 1918.16
Best individual: [1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0

In [12]:
simulation.simulate(simulation_count=25, p_c=0, p_m=1, population_size=200, iterations=100)

Simulation 1: best result - 1913
Simulation 2: best result - 1918
Simulation 3: best result - 1914
Simulation 4: best result - 1921
Simulation 5: best result - 1919
Simulation 6: best result - 1917
Simulation 7: best result - 1916
Simulation 8: best result - 1927
Simulation 9: best result - 1918
Simulation 10: best result - 1914
Simulation 11: best result - 1915
Simulation 12: best result - 1913
Simulation 13: best result - 1915
Simulation 14: best result - 1916
Simulation 15: best result - 1913
Simulation 16: best result - 1917
Simulation 17: best result - 1914
Simulation 18: best result - 1921
Simulation 19: best result - 1917
Simulation 20: best result - 1914
Simulation 21: best result - 1916
Simulation 22: best result - 1917
Simulation 23: best result - 1916
Simulation 24: best result - 1920
Simulation 25: best result - 1913

Best result: 1927
Mean: 1916.56
Best individual: [1 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0
 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0

Jak widać algorytm przy wyłączonym krzyżowaniem działa w nie najlepszy sposób, ale wciąż daje dodatni zysk.

Sprawdzam działanie algorytmu przy braku mutacji i przy zmianą prawdopodobieństwa krzyżowania.

In [13]:
simulation.simulate(simulation_count=25, p_c=0.1, p_m=0, population_size=200, iterations=100)

Simulation 1: best result - 1920
Simulation 2: best result - 1917
Simulation 3: best result - 1918
Simulation 4: best result - 1915
Simulation 5: best result - 1919
Simulation 6: best result - 1919
Simulation 7: best result - 1919
Simulation 8: best result - 1918
Simulation 9: best result - 1919
Simulation 10: best result - 1918
Simulation 11: best result - 1919
Simulation 12: best result - 1916
Simulation 13: best result - 1918
Simulation 14: best result - 1915
Simulation 15: best result - 1921
Simulation 16: best result - 1914
Simulation 17: best result - 1918
Simulation 18: best result - 1920
Simulation 19: best result - 1916
Simulation 20: best result - 1919
Simulation 21: best result - 1917
Simulation 22: best result - 1918
Simulation 23: best result - 1916
Simulation 24: best result - 1916
Simulation 25: best result - 1917

Best result: 1921
Mean: 1917.68
Best individual: [1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1

In [14]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0, population_size=200, iterations=100)

Simulation 1: best result - 1918
Simulation 2: best result - 1919
Simulation 3: best result - 1919
Simulation 4: best result - 1917
Simulation 5: best result - 1920
Simulation 6: best result - 1920
Simulation 7: best result - 1918
Simulation 8: best result - 1919
Simulation 9: best result - 1924
Simulation 10: best result - 1916
Simulation 11: best result - 1919
Simulation 12: best result - 1919
Simulation 13: best result - 1925
Simulation 14: best result - 1920
Simulation 15: best result - 1920
Simulation 16: best result - 1923
Simulation 17: best result - 1924
Simulation 18: best result - 1915
Simulation 19: best result - 1920
Simulation 20: best result - 1921
Simulation 21: best result - 1920
Simulation 22: best result - 1919
Simulation 23: best result - 1920
Simulation 24: best result - 1921
Simulation 25: best result - 1922

Best result: 1925
Mean: 1919.92
Best individual: [1 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0
 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0

In [24]:
simulation.simulate(simulation_count=25, p_c=1, p_m=0, population_size=200, iterations=100)

Simulation 1: best result - 1921
Simulation 2: best result - 1921
Simulation 3: best result - 1925
Simulation 4: best result - 1921
Simulation 5: best result - 1923
Simulation 6: best result - 1924
Simulation 7: best result - 1919
Simulation 8: best result - 1923
Simulation 9: best result - 1920
Simulation 10: best result - 1921
Simulation 11: best result - 1920
Simulation 12: best result - 1919
Simulation 13: best result - 1922
Simulation 14: best result - 1921
Simulation 15: best result - 1923
Simulation 16: best result - 1920
Simulation 17: best result - 1921
Simulation 18: best result - 1922
Simulation 19: best result - 1924
Simulation 20: best result - 1923
Simulation 21: best result - 1921
Simulation 22: best result - 1916
Simulation 23: best result - 1922
Simulation 24: best result - 1922
Simulation 25: best result - 1921

Best result: 1925
Mean: 1921.4
Best individual: [1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 1 0 1
 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 

W tej sytuacji algorytm zachowuje się tak samo, jak i przy włączonej mutacji, że przy wzroście prawdopodobieństwa krzyżowania, algorytm daje lepsze wyniki oraz średnia z eksperymentu rośnie.

## Wnioski

Po przeprowadzeniu powyższych eksperymentów mogę wywnioskować, że na działanie algorytmu ewolucyjnego głównie wpływają prawdopodobieństwa krzyżowania i mutacji oraz wielkości iteracji i populacji.
Każdy z tych parametrów decyduje o jakość wyniku. Żeby algorytm dał jak najlepszy wynik, musimy ustawić prawdopodobieństwo krzyżowania dość duże oraz małe prawdopodobieństwo mutacji. Co do populacji, to im więcej będzie osobników, tym lepiej działa algorytm, natomiast zbyt dużo osobników spowalnia proces optymalizacji. Analogicznie jest z wielkością iteracji.
Z powyższych eksperymentów wynika też, że dla prawdopodobieństwa krzyżowania równym 50%, prawdopodobieństwa mutacji - 5%, ilości populacji - 200 oraz ilości iteracji 100 daje jedne z najlepszych wyników w akceptowalny czas. Oczywiście zwiększenie wielkości populacji lub ilości iteracji polepszy wynik symulacji, ale wtedy będziemy mieli do czynienie z dość wolnym eksperymentem.


In [27]:
simulation.simulate(simulation_count=25, p_c=0.5, p_m=0.05, population_size=200, iterations=100)

Simulation 1: best result - 1922
Simulation 2: best result - 1924
Simulation 3: best result - 1923
Simulation 4: best result - 1923
Simulation 5: best result - 1920
Simulation 6: best result - 1924
Simulation 7: best result - 1921
Simulation 8: best result - 1923
Simulation 9: best result - 1922
Simulation 10: best result - 1922
Simulation 11: best result - 1922
Simulation 12: best result - 1925
Simulation 13: best result - 1921
Simulation 14: best result - 1923
Simulation 15: best result - 1925
Simulation 16: best result - 1923
Simulation 17: best result - 1920
Simulation 18: best result - 1922
Simulation 19: best result - 1921
Simulation 20: best result - 1921
Simulation 21: best result - 1923
Simulation 22: best result - 1924
Simulation 23: best result - 1921
Simulation 24: best result - 1923
Simulation 25: best result - 1925

Best result: 1925
Mean: 1922.52
Best individual: [1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0
 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0